<a href="https://colab.research.google.com/github/SinnottKayleigh/B2B-Sales-Algos/blob/main/MEDDPICC_Prospect_Scoring_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MEDDPICC test algo

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

np.random.seed(42)
n_samples = 1000

data = {
    'annual_revenue': np.random.uniform(100e6, 5e9, n_samples),
    'fx_volume': np.random.uniform(10e6, 500e6, n_samples),
    'intl_markets': np.random.randint(5, 50, n_samples),
    'current_fx_costs_pct': np.random.uniform(0.5, 2.0, n_samples),
    'potential_savings': np.random.uniform(100e3, 5e6, n_samples),

    'buyer_title': np.random.choice(['CFO', 'Treasurer', 'Head of Finance'], n_samples),
    'budget_authority': np.random.uniform(1e6, 10e6, n_samples),
    'fx_platform_experience': np.random.choice(['Yes', 'No'], n_samples),

    'tech_priority': np.random.randint(1, 6, n_samples),
    'cost_priority': np.random.randint(1, 6, n_samples),
    'risk_priority': np.random.randint(1, 6, n_samples),
    'compliance_level': np.random.choice(['High', 'Medium', 'Low'], n_samples),

    'manual_processing_hours': np.random.uniform(5, 40, n_samples),
    'current_provider_issues': np.random.randint(1, 6, n_samples),
    'fx_rate_transparency': np.random.choice(['Yes', 'No'], n_samples),
    'settlement_risk_concerns': np.random.choice(['Yes', 'No'], n_samples),

    'current_provider': np.random.choice(['Provider A', 'Provider B', 'Provider C', 'None'], n_samples),
    'contract_renewal_months': np.random.randint(1, 24, n_samples),
    'current_satisfaction': np.random.randint(1, 6, n_samples),


    'conversion_success': np.random.choice([0, 1], n_samples, p=[0.7, 0.3])

}

df = pd.DataFrame(data)

In [5]:
def calculate_metrics_score(row):
    score = 0
    # Higher scores for larger potential savings relative to FX volume
    savings_ratio = row['potential_savings'] / row['fx_volume']
    score += min(savings_ratio * 100, 5)  # Cap at 5 points

    # Higher scores for higher FX volumes
    volume_score = np.log10(row['fx_volume']) - 6  # Log scale starting at 1M
    score += min(volume_score, 5)

    return min(score, 10)  # Normalize to 10-point scale

def calculate_economic_buyer_score(row):
    score = 0
    # Points for senior titles
    title_scores = {'CFO': 5, 'Treasurer': 3, 'Head of Finance': 2}
    score += title_scores[row['buyer_title']]

    # Points for budget authority
    score += min(row['budget_authority'] / 1e6, 5)

    return min(score, 10)


In [10]:
def engineer_features(df):
    df['savings_ratio'] = df['potential_savings'] / df['fx_volume']
    df['cost_efficiency'] = df['current_fx_costs_pct'] * df['fx_volume']
    df['urgency_score'] = (6 - df['current_satisfaction']) * (24 - df['contract_renewal_months'])/24

    le = LabelEncoder()
    categorical_cols = ['buyer_title', 'fx_platform_experience', 'compliance_level',
                       'fx_rate_transparency', 'settlement_risk_concerns', 'current_provider']

    for col in categorical_cols:
        df[col + '_encoded'] = le.fit_transform(df[col])

    return df

def train_model(df):
    df = engineer_features(df)

    feature_cols = [
        'annual_revenue', 'fx_volume', 'intl_markets', 'current_fx_costs_pct',
        'potential_savings', 'budget_authority', 'tech_priority', 'cost_priority',
        'risk_priority', 'manual_processing_hours', 'current_provider_issues',
        'contract_renewal_months', 'current_satisfaction', 'savings_ratio',
        'cost_efficiency', 'urgency_score'
    ] + [col for col in df.columns if col.endswith('_encoded')]

    X = df[feature_cols]
    y = df['conversion_success']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train_scaled, y_train)

    return rf_model, scaler, feature_cols

def predict_probability(model, scaler, features, new_prospect):
    new_prospect_scaled = scaler.transform(new_prospect[features].reshape(1, -1))
    return model.predict_proba(new_prospect_scaled)[0][1]

In [17]:
def predict_probability(model, scaler, features, row):
    # Convert row to DataFrame if it's a Series
    if isinstance(row, pd.Series):
        row_df = pd.DataFrame([row])
    else:
        row_df = row

    # Select only the feature columns and convert to numpy array
    row_features = row_df[features].values

    # Scale the features
    row_scaled = scaler.transform(row_features)

    # Get probability
    return model.predict_proba(row_scaled)[0][1]

def calculate_final_score(row, model, scaler, features):
    # Calculate MEDDPICC component scores (simplified version)
    metrics_score = calculate_metrics_score(row)
    econ_buyer_score = calculate_economic_buyer_score(row)

    # Total MEDDPICC score (using just two components for simplicity)
    meddpicc_score = (metrics_score + econ_buyer_score) / 20  # Normalize to 0-1 scale

    # Get model probability
    model_prob = predict_probability(model, scaler, features, row)

    # Calculate final weighted score
    final_score = (0.4 * meddpicc_score + 0.6 * model_prob) * 100

    return final_score

def score_prospects(df):
    # Train the model
    model, scaler, features = train_model(df)

    # Calculate scores for each prospect
    scores = []
    for idx, row in df.iterrows():
        score = calculate_final_score(row, model, scaler, features)
        scores.append(score)

    # Add scores to dataframe
    df['final_score'] = scores

    # Sort by final score
    return df.sort_values('final_score', ascending=False)

# Run the scoring
scored_prospects = score_prospects(df)

# Display results
print("\nTop 10 Prospects by Score:")
print(scored_prospects[['annual_revenue', 'fx_volume', 'buyer_title', 'final_score']].head(10))

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ut


Top 10 Prospects by Score:
     annual_revenue     fx_volume      buyer_title  final_score
520    3.011241e+09  3.953102e+07              CFO    82.393876
886    3.298456e+09  1.457249e+07              CFO    80.927068
752    3.175769e+09  1.593918e+07              CFO    79.772414
177    1.995003e+09  6.242051e+07              CFO    79.190655
813    4.612125e+09  2.026303e+07              CFO    78.813409
672    4.931454e+08  4.353478e+08              CFO    78.389070
725    4.224961e+09  4.052298e+07              CFO    78.215403
159    3.668260e+09  2.433117e+07              CFO    77.889043
960    5.057135e+08  6.798473e+07              CFO    77.859852
746    1.981067e+09  1.940422e+07  Head of Finance    77.575792


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ut

In [19]:
import plotly.express as px
import plotly.graph_objects as go

def create_interactive_plot(df):
    df['hover_text'] = df.apply(lambda row:
        f"Company Revenue: ${row['annual_revenue']/1e6:.1f}M<br>" +
        f"FX Volume: ${row['fx_volume']/1e6:.1f}M<br>" +
        f"Buyer Title: {row['buyer_title']}<br>" +
        f"Final Score: {row['final_score']:.1f}<br>" +
        f"Current Provider: {row['current_provider']}", axis=1)

    fig = px.scatter(
        df,
        x='fx_volume',
        y='final_score',
        size='annual_revenue',
        color='buyer_title',
        hover_data=['current_provider'],
        custom_data=['hover_text'],
        title='Prospect Scoring Analysis',
        labels={
            'fx_volume': 'FX Trading Volume ($)',
            'final_score': 'Prospect Score',
            'buyer_title': 'Economic Buyer Title'
        }
    )

    fig.update_layout(
        hovermode='closest',
        plot_bgcolor='white',
        width=1000,
        height=600,
        legend_title_text='Economic Buyer Title',
        xaxis=dict(
            title='FX Trading Volume ($)',
            gridcolor='lightgrey',
            type='log',
            showgrid=True
        ),
        yaxis=dict(
            title='Prospect Score',
            gridcolor='lightgrey',
            showgrid=True
        )
    )

    fig.update_traces(
        hovertemplate="%{customdata[0]}<extra></extra>"
    )

    return fig

def create_score_distribution(df):
    fig = go.Figure()

    fig.add_trace(go.Histogram(
        x=df['final_score'],
        nbinsx=30,
        name='Score Distribution'
    ))

    fig.update_layout(
        title='Distribution of Prospect Scores',
        xaxis_title='Score',
        yaxis_title='Count',
        width=1000,
        height=400,
        showlegend=False
    )

    return fig

scatter_plot = create_interactive_plot(scored_prospects)
dist_plot = create_score_distribution(scored_prospects)

scatter_plot.show()
dist_plot.show()

print("\nScore Summary Statistics:")
print(scored_prospects['final_score'].describe())

def create_segments(df):
    df['segment'] = pd.cut(df['final_score'],
                          bins=[0, 25, 50, 75, 100],
                          labels=['Low Priority', 'Medium Priority',
                                 'High Priority', 'Top Priority'])

    segment_summary = df.groupby('segment').agg({
        'final_score': ['count', 'mean'],
        'fx_volume': 'mean',
        'annual_revenue': 'mean'
    }).round(2)

    return segment_summary

print("\nSegment Analysis:")
print(create_segments(scored_prospects))


Score Summary Statistics:
count    1000.000000
mean       41.084499
std        16.629385
min        16.749947
25%        28.698341
50%        34.260654
75%        53.106819
max        82.393876
Name: final_score, dtype: float64

Segment Analysis:
                final_score            fx_volume annual_revenue
                      count   mean          mean           mean
segment                                                        
Low Priority            101  22.66  2.844743e+08   2.604718e+09
Medium Priority         639  33.53  2.565134e+08   2.463206e+09
High Priority           235  65.69  2.664626e+08   2.518364e+09
Top Priority             25  77.25  1.270334e+08   2.935058e+09


<ipython-input-19-d719584ebb0c>:88: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

